## Module 10 - Markov Chain Monte Carlo (MCMC) Methods
<b>Markov Chains</b><br>
Markov Chains describe a series of possible events, where the probability of each event occurring depends on the event before it (and only the event before it). In the example below we create a function to predict weather at a given day given only the day-to-day transition probabilities.

In [1]:
import numpy as np
import random as rm
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from IPython.display import Image

In [2]:
# A simple Markov chain model for the weather in Python
# Adapted from http://www.cyber-omelette.com/2017/01/markov.html

# Possible sequences of events
transition_name = [["StoS","StoR"],["RtoR","RtoS"]]

# Probabilities matrix (transition matrix)
transition_matrix = [[0.9,0.1],[0.5,0.5]]

This function returns the weather prediction for a given day given the initial weather and the transition information.

In [3]:
def weather_forecast(days, transition_name, transition_matrix, weather_today):
    for day_ct in range(days):
        if weather_today == "Sunny":
            change = np.random.choice(transition_name[0], p=transition_matrix[0])
            if change == "StoS":
                pass
            else:
                weather_today = "Rainy"
        elif weather_today == "Rainy":
            change = np.random.choice(transition_name[1], p=transition_matrix[1])
            if change == "RtoR":
                pass
            else:
                weather_today = "Sunny"
        # time.sleep(0.2)
    return weather_today

# We forecast the weather for 100 days
weather_at_end = weather_forecast(100, transition_name, transition_matrix, "Sunny")
print(weather_at_end)

Rainy


We can take the function above and run it a number of times to create a simulation. This simulation results will tell us the proportion of times it's sunny on day 100.

In [4]:
def simulate_mc(num_simulations, num_days, starting_weather):
    # Now we'll run it 1000 times
    sunny_days = 0
    rainy_days = 0
    
    for run_ct in range(num_simulations):
        weather_result = weather_forecast(num_days, transition_name, transition_matrix, starting_weather)
        if weather_result == "Sunny":
            sunny_days += 1
        else:
            rainy_days += 1

    sunny_frac = float(sunny_days) / (float(sunny_days + rainy_days))
    return sunny_frac

frac = simulate_mc(1000, 100, "Rainy")
print("P(Sunny) at day 100 is %.4f" % frac)                        

P(Sunny) at day 100 is 0.8310


When we have enough days in our 'chain', the initial state is irrelevant. We can see how this looks by simulating for an increasing number of 'chain' days. Note, this step can take some time.

In [5]:
sunny_frac = []
rainy_frac = []

for i in range(25):
    sunny_frac.append(simulate_mc(1000, i, "Sunny"))
    rainy_frac.append(simulate_mc(1000, i, "Rainy"))

In [6]:
# Create traces
trace0 = go.Scatter(
    x = np.arange(25),
    y = sunny_frac,
    mode = 'lines',
    name = 'First Day Sunny'
)

trace1 = go.Scatter(
    x = np.arange(25),
    y = rainy_frac,
    mode = 'lines',
    name = 'First Day Rainy'
)

layout= go.Layout(
    xaxis= dict(title = 'Days'),
    yaxis=dict(title = 'P(Sunny)'),
)

data = [trace0, trace1]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

<b>Monte Carlo</b><br>
Monte Carlo methods rely on random sampling to produce numeric results. For example, the area of a semi-circle in one quadrant is given by:

$\frac{1}{4} \pi r^2$

and the area of the square is given by:

$r^2$

Therefore the ratio = $\pi /4$


A Monte Carlo method for determining $\pi$ using this knowledge works by

  - Drawing a set of random points from inside the unit square

  - Counting the number of points that fall inside the quarter of the unit circle

  - Calculating the ratio of the number of points inside the quarter of the unit circle to the total number of points, which approaches $\pi/4$ as the number of random points increases.
  

In [7]:
#Pi animation
Image(url='Pi_30K.gif')

### Bayesian method of estimating $\mu$
 
$$\overbrace{p(\mu \ |\ Data)}^{\text{posterior}} = \dfrac{\overbrace{p(Data \ | \ \mu)}^{\text{likelihood}} \cdot \overbrace{p(\mu)}^{\text{prior}}}{\underbrace{p(Data)}_{\text{marginal likelihood}}}$$

### The Mechanics of MCMC

The MCMC sampler draws parameter values from the prior distribution and computes the likelihood that the observed data came from a distribution with these parameter values. 

$$\overbrace{p(\mu \ |\ Data)}^{posterior} \varpropto \overbrace{p(Data \ | \ \mu)}^{likelihood} \cdot \overbrace{p(\mu)}^{prior}$$

This calculation acts as a guiding light for the MCMC sampler. As it draws values from the paramater priors, it computes the likelihood of these paramters given the data - and will try to guide the sampler towards areas of higher probability.

The Bayesian method is concerned with traversing and collect samples around the area of highest probability. All of the samples collected are considered to be a credible parameter.

In [8]:
Image(url='mcmc-animate.gif')

### References
- [MCMC animation by Maxwell Joeseph](http://blog.revolutionanalytics.com/2013/09/an-animated-peek-into-the-workings-of-Bayesian-statistics.html)